# has performance

In [84]:
import ipytest

ipytest.autoconfig(raise_on_error=True)  # For CI environments

In [85]:
import duckdb


class HasPerformance:
    def __init__(self):
        self.con = duckdb.connect(":memory:")
        self.con.sql(
            """
            CREATE SEQUENCE id_sequence START 1;
            CREATE TABLE tbl (id INTEGER DEFAULT nextval('id_sequence'), created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, s VARCHAR);
            """
        )

    def save(self, txt: str):
        """save some txt

        Args:
            txt (str): any txt
        """
        self.con.sql(f"INSERT INTO tbl (s) VALUES ('{txt}');")

In [86]:
import pytest


@pytest.fixture(scope="session", autouse=True)
def get_hp():
    return HasPerformance()

In [92]:
%%ipytest -vv -s

def test(get_hp):
    get_hp.save('abc')
    get_hp.save('efg')

    res = "".join(str(get_hp.con.sql("SELECT id, s FROM tbl").fetchall()))

    assert res == "[(1, 'abc'), (2, 'efg')]"
    print("".join(str(get_hp.con.sql("SELECT * FROM tbl").fetchall())))


======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.2.1, pluggy-1.5.0 -- /home/xsazcd/git/obar1/py-tiny-datastructures.git/.venv/bin/python
cachedir: .pytest_cache
rootdir: /home/xsazcd/git/obar1/py-tiny-datastructures.git/has-performance
plugins: cov-4.0.0, pytest_notebook-0.10.0, nbval-0.10.0, anyio-4.8.0
collecting ... collected 1 item

t_97bf78ae14a243e2aa476ddf606d42ed.py::test collected 1 item

t_97bf78ae14a243e2aa476ddf606d42ed.py::test [(1, datetime.datetime(2025, 3, 14, 17, 36, 49, 452000), 'abc'), (2, datetime.datetime(2025, 3, 14, 17, 36, 49, 453000), 'efg')]
PASSED

======================================== 1 passed in 0.03s =========================================
